<a href="https://colab.research.google.com/github/anilkumarpanda/Santander-Customer-Transaction-Prediction/blob/master/Outlier_Detection_PyOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyod

    100% |████████████████████████████████| 81kB 3.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/af/5b/a8/db41636737f746250b4134e9a12a95aa24feef4ecc3e871b0b
Successfully built pyod


In [0]:
# Import models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc


In [11]:
print(os.listdir("data/"))
# import Dataset to play with it
train= pd.read_csv("data/train.csv")
test = pd.read_csv('data/test.csv')

['train.csv', 'sample_submission.csv', 'test.csv']


In [12]:
#Divide into train and test
from pandas import read_csv
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler
cols=["target","ID_code"]
X = train.drop(cols,axis=1)
y = train["target"]
X_test  = test.drop("ID_code",axis=1)

validation_size = 0.20
seed = 33

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(rescaledX,y,test_size=validation_size,random_state=seed)
X_train.shape,X_test.shape

((160000, 200), (40000, 200))

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4).fit(rescaledX)
pca_nx = pca.transform(rescaledX)

In [48]:
pca_df = pd.DataFrame(data=pca_nx,columns=['pca_1','pca_2','pca_3','pca_4'])

pca_df.head()

,pca_1,pca_2,pca_3,pca_4
0,0.018013664164505,-0.025755592466444,-0.415965158128451,-0.044392308248081
1,0.248578723545490,-0.196295182480967,-0.025412630839243,-0.050199374050031
2,0.195983016901804,-0.112595104500651,-0.224711410179849,-0.219370248498425
3,-0.202067883392939,0.086090368627794,0.273961892881527,-0.220738169818783
4,-0.184335456774243,0.200632568889781,-0.040996052657104,0.094185400977954


In [66]:
# Prepare Data for Isolatation Forests
pca_df = pd.DataFrame(data=pca_nx,columns=['pca_1','pca_2','pca_3','pca_4'])
pca_df.head()
pca_df['target']=y

y_0 = train.loc[train['target']==0]
y_0 = y_0['target']
print(y_0.shape)

X_isf_0 =  pca_df.loc[pca_df['target']==0]
X_isf_0 = X_isf_0.drop('target',axis=1)
print(X_isf_0.shape)
X_isf_1 =  pca_df.loc[pca_df['target']==1]
X_isf_1 = X_isf_1.drop('target',axis=1)




(179902,)
(179902, 4)


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

rng = np.random.RandomState(42)

X_train, X_test, y_train, y_test = train_test_split(X_isf_0,y_0,test_size=validation_size,random_state=seed)

# #X_train.shape,X_test.shape
# # Generate train data
# X = 0.3 * rng.randn(100, 2)
# X_train = np.r_[X + 2, X - 2]
# # Generate some regular novel observations
# X = 0.3 * rng.randn(20, 2)
# X_test = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = X_isf_1

# fit the model
clf = IsolationForest(behaviour='new', max_samples=100,
                      random_state=rng, contamination='auto')
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

# # plot the line, the samples, and the nearest vectors to the plane
# xx, yy = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50))
# Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
# Z = Z.reshape(xx.shape)

# plt.title("IsolationForest")
# plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

# b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white',s=20, edgecolor='k')
# b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='green',s=20, edgecolor='k')
# c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='red',s=20, edgecolor='k')

# plt.axis('tight')
# plt.xlim((-5, 5))
# plt.ylim((-5, 5))
# plt.legend([b1, b2, c],
#            ["training observations",
#             "new regular observations", "new abnormal observations"],
#            loc="upper left")
# plt.show()

In [71]:
print(y_pred_train.shape)
print(y_pred_test.shape)
print(y_pred_outliers.shape)

prediction_df = pd.DataFrame()

prediction_df['y_pred_train'] = y_pred_train
prediction_df['y_train'] = y_train

prediction_df['y_pred_train'].value_counts()
prediction_df['y_train'].value_counts()

(143921,)
(35981,)
(20098,)


0.0    103556
Name: y_train, dtype: int64

In [0]:
# params is based on following kernel https://www.kaggle.com/brandenkmurray/nothing-works
params = {'objective' : "binary", 
               'boost':"gbdt",
               'metric':"auc",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01,
               'num_leaves' : 13,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 0.05,
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4,
               'min_data_in_leaf' : 80,
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

In [0]:
# for get better result chage fold_n to 5
fold_n=5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

%time
y_pred_lgb = np.zeros(len(X_test))
num_round = 1000000
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params,train_data,num_round,#change 20 to 2000
                    valid_sets = [train_data, valid_data],verbose_eval=1000,early_stopping_rounds = 3500)##change 10 to 200
            
    y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)/5